### Trying to converti the csv using rdd

In [23]:
from pyspark.sql import SparkSession

In [24]:
spark = SparkSession.builder \
    .appName("Structure datasett and convert to csv") \
    .getOrCreate()

In [25]:
csv_path = "hdfs:///eaplayer_data/male_players.csv"

In [26]:
df = spark.read.option("header", "true").csv(csv_path)

In [27]:
df.take(2)

[Row(player_id='158023', player_url='/player/158023/lionel-messi/230009', fifa_version='23', fifa_update='9', fifa_update_date='2023-01-13', short_name='L. Messi', long_name='Lionel Andrés Messi Cuccittini', player_positions='RW', overall='91', potential='91', value_eur='54000000', wage_eur='195000', age='35', dob='1987-06-24', height_cm='169', weight_kg='67', league_id='16', league_name='Ligue 1', league_level='1', club_team_id='73', club_name='Paris Saint Germain', club_position='RS', club_jersey_number='30', club_loaned_from=None, club_joined_date='2021-08-10', club_contract_valid_until_year='2023', nationality_id='52', nationality_name='Argentina', nation_team_id='1369', nation_position='RW', nation_jersey_number='10', preferred_foot='Left', weak_foot='4', skill_moves='4', international_reputation='5', work_rate='Low/Low', body_type='Unique', real_face='Yes', release_clause_eur='99900000', player_tags='#Dribbler, #Distance Shooter, #FK Specialist, #Acrobat, #Clinical Finisher, #Com

### Checking for 'none' values. (Debugging after i found them while cleaning.)
- turns out every single one row has a none value

In [28]:
rows_with_none_count = df.filter(
    " OR ".join([f"({c} IS NULL OR {c} = '' OR isnan({c}))" for c in df.columns])
).count()
rows_with_none_count

10003590

In [29]:
header = ",".join(df.columns)
header

'player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_intercepti

Converting each row of the DataFrame into a Spark Row object within an RDD.

In [30]:
rdd = df.rdd

In [31]:
rdd.first()

Row(player_id='158023', player_url='/player/158023/lionel-messi/230009', fifa_version='23', fifa_update='9', fifa_update_date='2023-01-13', short_name='L. Messi', long_name='Lionel Andrés Messi Cuccittini', player_positions='RW', overall='91', potential='91', value_eur='54000000', wage_eur='195000', age='35', dob='1987-06-24', height_cm='169', weight_kg='67', league_id='16', league_name='Ligue 1', league_level='1', club_team_id='73', club_name='Paris Saint Germain', club_position='RS', club_jersey_number='30', club_loaned_from=None, club_joined_date='2021-08-10', club_contract_valid_until_year='2023', nationality_id='52', nationality_name='Argentina', nation_team_id='1369', nation_position='RW', nation_jersey_number='10', preferred_foot='Left', weak_foot='4', skill_moves='4', international_reputation='5', work_rate='Low/Low', body_type='Unique', real_face='Yes', release_clause_eur='99900000', player_tags='#Dribbler, #Distance Shooter, #FK Specialist, #Acrobat, #Clinical Finisher, #Comp

Formatting each row

In [32]:
def format_row(row):
    return ', '.join([str(field).replace(',', ';') for field in row])


In [33]:
formatted_rdd = rdd.map(format_row)

In [34]:
formatted_rdd.first()

'158023, /player/158023/lionel-messi/230009, 23, 9, 2023-01-13, L. Messi, Lionel Andrés Messi Cuccittini, RW, 91, 91, 54000000, 195000, 35, 1987-06-24, 169, 67, 16, Ligue 1, 1, 73, Paris Saint Germain, RS, 30, None, 2021-08-10, 2023, 52, Argentina, 1369, RW, 10, Left, 4, 4, 5, Low/Low, Unique, Yes, 99900000, #Dribbler; #Distance Shooter; #FK Specialist; #Acrobat; #Clinical Finisher; #Complete Forward, Finesse Shot; Long Shot Taker (AI); Playmaker (AI); Outside Foot Shot; Chip Shot (AI); Technical Dribbler (AI), 81, 89, 90, 94, 34, 64, 84, 90, 70, 91, 88, 95, 93, 93, 90, 93, 87, 76, 91, 92, 95, 86, 68, 70, 68, 91, 44, 40, 93, 94, 75, 96, 20, 35, 24, 6, 11, 15, 14, 8, None, 87+3, 87+3, 87+3, 90, 91, 91, 91, 90, 92-1, 92-1, 92-1, 89+2, 85+3, 85+3, 85+3, 89+2, 64+3, 63+3, 63+3, 63+3, 64+3, 59+3, 50+3, 50+3, 50+3, 59+3, 19+3, https://cdn.sofifa.net/players/158/023/23_120.png'

In [35]:
def has_none_values(fields):
    return any(field.strip() == "" or field.strip().lower() == "none" for field in fields)
none_rows = formatted_rdd.filter(has_none_values)
none_count = none_rows.count()
print(f"Number of rows with None values: {none_count}")

Number of rows with None values: 10003590


In [36]:
formatted_rdd.take(5)


['158023, /player/158023/lionel-messi/230009, 23, 9, 2023-01-13, L. Messi, Lionel Andrés Messi Cuccittini, RW, 91, 91, 54000000, 195000, 35, 1987-06-24, 169, 67, 16, Ligue 1, 1, 73, Paris Saint Germain, RS, 30, None, 2021-08-10, 2023, 52, Argentina, 1369, RW, 10, Left, 4, 4, 5, Low/Low, Unique, Yes, 99900000, #Dribbler; #Distance Shooter; #FK Specialist; #Acrobat; #Clinical Finisher; #Complete Forward, Finesse Shot; Long Shot Taker (AI); Playmaker (AI); Outside Foot Shot; Chip Shot (AI); Technical Dribbler (AI), 81, 89, 90, 94, 34, 64, 84, 90, 70, 91, 88, 95, 93, 93, 90, 93, 87, 76, 91, 92, 95, 86, 68, 70, 68, 91, 44, 40, 93, 94, 75, 96, 20, 35, 24, 6, 11, 15, 14, 8, None, 87+3, 87+3, 87+3, 90, 91, 91, 91, 90, 92-1, 92-1, 92-1, 89+2, 85+3, 85+3, 85+3, 89+2, 64+3, 63+3, 63+3, 63+3, 64+3, 59+3, 50+3, 50+3, 50+3, 59+3, 19+3, https://cdn.sofifa.net/players/158/023/23_120.png',
 '165153, /player/165153/karim-benzema/230009, 23, 9, 2023-01-13, K. Benzema, Karim Benzema, CF; ST, 91, 91, 64000

In [37]:
""" header = formatted_rdd.filter(lambda row: "player_id" in row).first() """

' header = formatted_rdd.filter(lambda row: "player_id" in row).first() '

In [38]:
header_rdd = spark.sparkContext.parallelize([header])
final_rdd = header_rdd.union(formatted_rdd)
final_rdd.first()

'player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_intercepti

In [39]:
final_rdd.take(5)

['player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_intercept

Counting Columns

In [40]:
column_counts = final_rdd.map(lambda line: len(line.split(',')))

In [41]:
column_counts.take(5)


[110, 110, 110, 110, 110]

In [ ]:
formatted_rdd = final_rdd.repartition(50) 


' formatted_rdd = final_rdd.repartition(50) \n '

In [ ]:
# formatted_rdd.saveAsTextFile("hdfs:///eaplayer_data/male_players_text_revised")

In [44]:
spark.stop()

- About data (10 million rows aprox)
- 10003590 rows